### Imports

In [2]:
import pandas as pd
from transformers import (
    AutoTokenizer, 
    AutoModelForPreTraining, 
    AutoModel,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding, 
    Trainer,
    TrainingArguments
)
from datasets import load_dataset, Dataset

/home/mateuslatrova/miniconda3/envs/enem/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Variáveis "globais"

In [4]:
checkpoint = 'neuralmind/bert-base-portuguese-cased'

## Exploração do dataset

### Carregamento e primeiro contato com os dados

In [3]:
# Colunas do dataset:
#
# id = identificador da redacao
# id_prompt = tema da redacao talvez?
# title = titulo da redacao
# essay = texto da redacao
# grades = notas

dataset = load_dataset("csv", data_files="data/DatasetGrande.csv")

train_dataset = dataset["train"]
train_dataset.set_format("pandas")
train_dataset_df = train_dataset[:]
train_dataset_df.head()

,id,id_prompt,title,essay,grades,general,specific
0,41,1,...,"['As animações japonesas, nomeadas ""Anime"", po...","[160, 160, 120, 200, 160, 0]",NaN,NaN
1,42,1,1,['Vive-se em uma sociedade erotização infantil...,"[160, 160, 120, 120, 160, 0]",NaN,NaN
2,43,1,A cultura da erotização infantil,['A sexualização precoce tem impactado cada ve...,"[200, 120, 120, 200, 160, 0]",NaN,NaN
3,44,1,Pequenos Corpos Desrespeitados,['O ser humano passa por dois estágios até ati...,"[160, 160, 120, 200, 160, 0]",NaN,NaN
4,45,1,"Pequenos em Tamanho, Grandes em Problemas",['Em um mundo globalizado e dinamicamente tecn...,"[200, 160, 160, 120, 160, 0]",NaN,NaN


### Validando o formato das redações

In [4]:
samples = train_dataset.shuffle(7).select(range(3))
samples.set_format("pandas")
samples_df = samples[:]
essay_examples = samples_df["essay"].tolist()
essay_examples

["['A tecnologia está muito presente na sociedade, principalmente entre as crianças. Porém, esse uso excessivo acaba prejudicando no desenvolvimento da criança, além de atrapalhar as relações, principalmente familiar. Nessa perspectiva, faz-se importante analisar, os fatores que favorecem esse quadro.', 'Para compreender esse cenário, deve-se ressaltar a ausência de limites, em relação ao tempo de uso das redes, parar combater o problema. Nesse sentido, esse constante acesso à internet pode gerar consequências criança, como aponta a OMS . Essa situação, segundo a OMS, pode levar à inatividade física e prejudicar o desenvolvimento cognitivo.', 'Além disso, é fundamental apontar a sensação de poder e irresponsabilidade como propulsor do problema. Segundo, cerca de 69% das crianças e adolescentes que têm entre 9 e 17 anos e com acesso à internet a ultilizam mais do que uma vez por dia. Diante disso, so 69% de crianças e adolescentes expostos a todo momento a essa sensação de poder e irres

In [6]:
# Como há apenas valores "True" nessa máscara, todas as redações iniciam com '[' e terminam com ']'.
mask = (train_dataset_df["essay"].str.startswith("[")) & (train_dataset_df["essay"].str.endswith("]"))
set(mask.values.tolist())

{True}

In [7]:
# Para remover esses caracteres de início e fim:
for example  in essay_examples:
    print(example.strip("['']"))

A tecnologia está muito presente na sociedade, principalmente entre as crianças. Porém, esse uso excessivo acaba prejudicando no desenvolvimento da criança, além de atrapalhar as relações, principalmente familiar. Nessa perspectiva, faz-se importante analisar, os fatores que favorecem esse quadro.', 'Para compreender esse cenário, deve-se ressaltar a ausência de limites, em relação ao tempo de uso das redes, parar combater o problema. Nesse sentido, esse constante acesso à internet pode gerar consequências criança, como aponta a OMS . Essa situação, segundo a OMS, pode levar à inatividade física e prejudicar o desenvolvimento cognitivo.', 'Além disso, é fundamental apontar a sensação de poder e irresponsabilidade como propulsor do problema. Segundo, cerca de 69% das crianças e adolescentes que têm entre 9 e 17 anos e com acesso à internet a ultilizam mais do que uma vez por dia. Diante disso, so 69% de crianças e adolescentes expostos a todo momento a essa sensação de poder e irrespons

In [9]:
# Além disso, aparentemente, os parágrafos da redação são iniciados e terminados pelos caracteres "\'" ou apenas "'".
# Para validar isso, faremos o seguinte teste:
for example  in essay_examples:
    example = example.strip("['']")
    removed_quote_and_comma = example.replace("\',", "")
    removed_quote = removed_quote_and_comma.replace("\'", "")
    removed_slash = removed_quote.replace("\\", "")
    print(removed_slash)

A tecnologia está muito presente na sociedade, principalmente entre as crianças. Porém, esse uso excessivo acaba prejudicando no desenvolvimento da criança, além de atrapalhar as relações, principalmente familiar. Nessa perspectiva, faz-se importante analisar, os fatores que favorecem esse quadro. Para compreender esse cenário, deve-se ressaltar a ausência de limites, em relação ao tempo de uso das redes, parar combater o problema. Nesse sentido, esse constante acesso à internet pode gerar consequências criança, como aponta a OMS . Essa situação, segundo a OMS, pode levar à inatividade física e prejudicar o desenvolvimento cognitivo. Além disso, é fundamental apontar a sensação de poder e irresponsabilidade como propulsor do problema. Segundo, cerca de 69% das crianças e adolescentes que têm entre 9 e 17 anos e com acesso à internet a ultilizam mais do que uma vez por dia. Diante disso, so 69% de crianças e adolescentes expostos a todo momento a essa sensação de poder e irresponsabilid

In [11]:
# Um jeito mais eficiente que descobri para realizar a mesma tarefa:
for example in essay_examples:
    essay = ""
    for paragraph in eval(example):
        essay += paragraph
    print(essay)

A tecnologia está muito presente na sociedade, principalmente entre as crianças. Porém, esse uso excessivo acaba prejudicando no desenvolvimento da criança, além de atrapalhar as relações, principalmente familiar. Nessa perspectiva, faz-se importante analisar, os fatores que favorecem esse quadro.Para compreender esse cenário, deve-se ressaltar a ausência de limites, em relação ao tempo de uso das redes, parar combater o problema. Nesse sentido, esse constante acesso à internet pode gerar consequências criança, como aponta a OMS . Essa situação, segundo a OMS, pode levar à inatividade física e prejudicar o desenvolvimento cognitivo.Além disso, é fundamental apontar a sensação de poder e irresponsabilidade como propulsor do problema. Segundo, cerca de 69% das crianças e adolescentes que têm entre 9 e 17 anos e com acesso à internet a ultilizam mais do que uma vez por dia. Diante disso, so 69% de crianças e adolescentes expostos a todo momento a essa sensação de poder e irresponsabilidad

### Validando o que significa o campo "id_prompt"

In [10]:
# Validando se o campo "id_prompt" é de fato um identificador para o tema da redação
id_prompt_to_count = train_dataset_df["id_prompt"].value_counts().to_dict()
list(id_prompt_to_count.items())[0:10]

[(28, 121),
 (1, 120),
 (68, 120),
 (14, 119),
 (53, 118),
 (55, 118),
 (2, 118),
 (66, 117),
 (25, 117),
 (44, 117)]

In [11]:
# Observando o dataframe abaixo, podemos concluir que, de fato, o campo "id_prompt" é um identificador para 
# o tema da redação.
train_dataset_df[train_dataset_df["id_prompt"] == 1].sample(frac=1).head(10)

,id,id_prompt,title,essay,grades,general,specific
35,76,1,A erotização infantil,['A erotização infantil vem se tornando uma si...,"[160, 120, 80, 120, 120, 0]",NaN,NaN
67,108,1,Crianças adultas,['A sexualidade é algo comum e muito natural p...,"[160, 120, 120, 120, 120, 0]",NaN,NaN
14,55,1,Teste da redação 22,['Em um dos episódios da série “Law And Order ...,"[160, 200, 160, 200, 200, 0]",NaN,NaN
83,124,1,Erotização infantil,['A erotização infantil é caracterizada pela i...,"[160, 160, 160, 200, 160, 0]",NaN,NaN
7,48,1,REDAÇÃO,['Podemos ver na série americana ``Sex Educati...,"[160, 160, 120, 120, 160, 0]",NaN,NaN
90,131,1,Fatores que influenciam a erotização infantil,['Muito se é discutido a respeito da erotizaçã...,"[120, 120, 120, 120, 120, 0]",NaN,NaN
45,86,1,A infância precisa ser vivida.,"['""Cuties"", filme recente da Netflix, foi alvo...","[160, 120, 120, 120, 120, 0]",NaN,NaN
96,137,1,Infância em perigo,['A erotização infantil burla a etapa da infân...,"[160, 120, 120, 160, 120, 0]",NaN,NaN
99,140,1,Infância Roubada,"['""É típica a conduta de fotografar cenas porn...","[160, 120, 120, 120, 160, 0]",NaN,NaN
52,93,1,A sensualização de crianças e a normalidade di...,"['Desde o início da era das redes sociais, inú...","[160, 160, 120, 120, 160, 0]",NaN,NaN


### Validando remoção de colunas sem dados

In [12]:
# Como todos valores dessas duas colunas são nulos, podemos removê-las.
columns_to_be_removed = ["general", "specific"]
for column in columns_to_be_removed:
    print(all(train_dataset_df[column].isna()))

True
True


### Validando se é possível normalizar o campo "title"

In [13]:
title_to_count = train_dataset_df["title"].value_counts()
title_to_count = {title : count for title, count in title_to_count.items() if count > 1}
title_to_count

{'[Sem título]': 616,
 '.': 14,
 '12080': 13,
 '...': 12,
 'uma nova era': 12,
 'Consequências do etarismo no Brasil': 11,
 'Lixo têxtil: uma ameaça ao meio ambiente': 10,
 'Dependentes químicos: desafios para o tratamento adequado no Brasil': 10,
 'O aumento da circulação de armas de fogo no Brasil': 10,
 'Aumento do alcoolismo entre as mulheres': 9,
 'Impactos da polarização política no Brasil': 9,
 'Insegurança alimentar no Brasil': 8,
 'Autismo: desafios para a inclusão': 8,
 'PEC 206: um retrocesso na educação?': 8,
 'Mercado de trabalho: desafios enfrentados por pessoas com deficiência': 7,
 'A cultura da erotização infantil': 7,
 'A importância da participação da mulher na política': 7,
 'O estigma associado às doenças mentais na sociedade brasileira': 7,
 '4587A': 7,
 'P': 6,
 'Ariane R.': 6,
 'Valorização do trabalho docente no Brasil': 6,
 'Parto humanizado': 6,
 'O uso de tecnologia entre as crianças': 5,
 'REDAÇÃO': 5,
 'Sem título': 5,
 'Efeitos do negacionismo científico 

In [14]:
# Depois de analisar todos esses títulos estranhos, concluiu-se que todos eles são títulos
# inválidos, ou são simplesmente um valor indicando que a redação não teve título.
suspicious_titles = list(title_to_count.keys())
train_dataset_df[train_dataset_df["title"].isin(suspicious_titles)].sort_values(by=["title"])

,id,id_prompt,title,essay,grades,general,specific
1278,3071,32,#Princesa redatora,"['No ano de 1889, o filósofo brasileiro Raimun...","[160, 160, 120, 200, 160, 0]",NaN,NaN
255,408,4,#Princesa redatora,['A participação feminina na política brasilei...,"[160, 160, 160, 200, 200, 0]",NaN,NaN
434,1224,13,#Princesa redatora,"['Durante a década dos anos 1920, nos Unidos, ...","[160, 160, 160, 200, 200, 0]",NaN,NaN
1589,4118,44,**,"['Com a globalização, os fluxos de informação ...","[160, 200, 160, 200, 160, 0]",NaN,NaN
1578,4107,44,**,"['""O Dilema das Redes"", documentário produzido...","[160, 200, 160, 200, 160, 0]",NaN,NaN
...,...,...,...,...,...,...,...
1941,4821,51,xxx,"['A série “Gossip Girl”, transmitida pela rede...","[160, 160, 160, 200, 200, 0]",NaN,NaN
2076,5007,55,xxx,['A Segunda Guerra Mundial foi um período marc...,"[160, 200, 160, 200, 200, 0]",NaN,NaN
2724,6329,68,xxx,['A Guerra das Rosas foi um conflito interno n...,"[160, 160, 120, 200, 200, 0]",NaN,NaN
2921,7356,79,xxx,['A Revolução Industrial foi um período de gra...,"[200, 160, 120, 200, 200, 0]",NaN,NaN


In [17]:
# Portanto, vamos apenas colocar um valor nulo para todos esses títulos inválidos
train_dataset_df.loc[train_dataset_df["title"].isin(suspicious_titles), "title"] = pd.NA
train_dataset_df[train_dataset_df["title"].isin(suspicious_titles)]

,id,id_prompt,title,essay,grades,general,specific


In [18]:
train_dataset_df["title"].unique().tolist()

[<NA>,
 '1',
 'Pequenos Corpos Desrespeitados',
 'Pequenos em Tamanho, Grandes em Problemas',
 'Pierre Bourdieu redação',
 'Preciosa Infância',
 'Sexualização infantil',
 'Sexualização Infantil',
 'sexualização na infância',
 'sexualização precoce',
 'Suas decisões na vida de uma criança.',
 'Teste da redação 22',
 'Um dever dos adultos com as crianças',
 'A Cultura da Erotização Infantil',
 'A cultura da erotização infantil e sua precocidade',
 'A cultura da erotização infantil: um forte apelo midiático e social',
 'A desmoralização sexual na infância',
 'A erotização de crianças',
 'A Erotização Infantil',
 'A erotização infantil e os problemas que isso acarreta',
 'A EROTIZAÇÃO PREMATURA (VI)',
 'A evolução da infância',
 'A infância como um bem social',
 'A Infância e o Amadurecimento Precoce',
 'A adultização  infantil',
 'A infância oprimida',
 'A infância precisa ser vivida.',
 'A Influência da Mídia Sobre a Erotização Infantil',
 'A inocência de uma criança',
 'A normalização d

Porém, depois de analisar mais exemplos, percebeu-se que há mais títulos inválidos. Então, caso seja realmente necessário remover esses títulos inválidos para alguma tarefa, isso será realizado posteriormente devido ao grande número de exemplos.

### Análise do campo "grades"

In [13]:
samples = train_dataset.shuffle(7).select(range(3))
samples.set_format("pandas")
samples_df = samples[:]
grades_examples = samples_df["grades"].tolist()
grades_examples

['[160, 160, 160, 200, 160, 0]',
 '[160, 160, 120, 160, 120, 0]',
 '[160, 160, 120, 200, 120, 0]']

In [14]:
type(grades_examples[0])

str

In [15]:
example = grades_examples[0]
grades_without_brackets = example.strip('[]')
grades_without_brackets

'160, 160, 160, 200, 160, 0'

In [16]:
# Obtendo a nota de cada competência separadamente, podemos criar uma feature de nota por competência, sendo do tipo inteiro.
[int(grade) for grade in grades_without_brackets.split(",")]

[160, 160, 160, 200, 160, 0]

In [17]:
# Uma forma mais eficiente de realizar a mesma tarefa acima:
for example in grades_examples:
    print(eval(example))

[160, 160, 160, 200, 160, 0]
[160, 160, 120, 160, 120, 0]
[160, 160, 120, 200, 120, 0]


## Dataset preprocessing

In [48]:
def preprocess_essay(example):
    essay_before = example["essay"]
    essay_after = ""
    paragraphs = eval(essay_before)
    for paragraph in paragraphs:
        essay_after += paragraph
    example["essay_text"] = essay_after
    return example

def create_column_for_second_grade(example):
    second_grade_index = 1
    grades = eval(example["grades"])
    example["second_grade"] = grades[second_grade_index]
    return example

def preprocess_dataset(dataset: Dataset):
    dataset = dataset.remove_columns(["general", "specific"])
    dataset = dataset.rename_columns({"id": "essay_id", "id_prompt": "topic_id"})
    dataset = dataset.map(preprocess_essay).map(create_column_for_second_grade)
    dataset = dataset.remove_columns(["grades", "essay"])
    return dataset


In [49]:
dataset = load_dataset("csv", data_files="data/DatasetGrande.csv")

train_dataset = dataset["train"]
train_dataset = preprocess_dataset(train_dataset)

train_dataset

Map: 100%|██████████| 3219/3219 [00:01<00:00, 1928.28 examples/s]


Dataset({
    features: ['essay_id', 'topic_id', 'title', 'essay_text', 'second_grade'],
    num_rows: 3219
})

## Example of an essay tokenization

In [50]:
essay_samples = train_dataset.shuffle(23).select(range(3))
essay_samples["essay_text"]
for example in essay_examples:
    print(example)

['A tecnologia está muito presente na sociedade, principalmente entre as crianças. Porém, esse uso excessivo acaba prejudicando no desenvolvimento da criança, além de atrapalhar as relações, principalmente familiar. Nessa perspectiva, faz-se importante analisar, os fatores que favorecem esse quadro.', 'Para compreender esse cenário, deve-se ressaltar a ausência de limites, em relação ao tempo de uso das redes, parar combater o problema. Nesse sentido, esse constante acesso à internet pode gerar consequências criança, como aponta a OMS . Essa situação, segundo a OMS, pode levar à inatividade física e prejudicar o desenvolvimento cognitivo.', 'Além disso, é fundamental apontar a sensação de poder e irresponsabilidade como propulsor do problema. Segundo, cerca de 69% das crianças e adolescentes que têm entre 9 e 17 anos e com acesso à internet a ultilizam mais do que uma vez por dia. Diante disso, so 69% de crianças e adolescentes expostos a todo momento a essa sensação de poder e irrespo

In [23]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint, do_lower_case=False)
tokenizer

BertTokenizerFast(name_or_path='neuralmind/bert-base-portuguese-cased', vocab_size=29794, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [24]:
tokenized_essays = []
for essay in essay_examples:
    tokenized_essay = tokenizer.tokenize([essay], is_split_into_words=True)
    tokenized_essays.append(tokenized_essay)
tokenized_essays[0]

['[',
 "'",
 'A',
 'tecnologia',
 'está',
 'muito',
 'presente',
 'na',
 'sociedade',
 ',',
 'principalmente',
 'entre',
 'as',
 'crianças',
 '.',
 'Porém',
 ',',
 'esse',
 'uso',
 'excess',
 '##ivo',
 'acaba',
 'prejudic',
 '##ando',
 'no',
 'desenvolvimento',
 'da',
 'criança',
 ',',
 'além',
 'de',
 'atrapal',
 '##har',
 'as',
 'relações',
 ',',
 'principalmente',
 'familiar',
 '.',
 'Nessa',
 'perspectiva',
 ',',
 'faz',
 '-',
 'se',
 'importante',
 'analisar',
 ',',
 'os',
 'fatores',
 'que',
 'favore',
 '##cem',
 'esse',
 'quadro',
 '.',
 "'",
 ',',
 "'",
 'Para',
 'compreender',
 'esse',
 'cenário',
 ',',
 'deve',
 '-',
 'se',
 'ressaltar',
 'a',
 'ausência',
 'de',
 'limites',
 ',',
 'em',
 'relação',
 'ao',
 'tempo',
 'de',
 'uso',
 'das',
 'redes',
 ',',
 'parar',
 'combater',
 'o',
 'problema',
 '.',
 'Nesse',
 'sentido',
 ',',
 'esse',
 'constante',
 'acesso',
 'à',
 'internet',
 'pode',
 'gerar',
 'consequências',
 'criança',
 ',',
 'como',
 'aponta',
 'a',
 'O',
 '##MS',


## Example of a model fine-tuning

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:


training_args = TrainingArguments("test-trainer")

In [ ]:
from transformers import 

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()